In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pickle

In [3]:
zomato = pd.read_csv('zomato.csv', encoding='latin-1')

In [3]:
baseReduzida = zomato

In [4]:
baseReduzida.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [10]:
baseReduzida = baseReduzida[~baseReduzida["cozinha"].str.contains(" ", na=False)]
baseReduzida.shape

(1679, 21)

In [14]:
baseReduzida.to_csv('basereduzida.csv', index=False)

In [15]:
t = pd.read_csv('basereduzida.csv', encoding='latin-1')

In [22]:
zomato.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [23]:
zomato.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,cozinha,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,alcance_preco,classifi_agregada,Rating color,taxa_votos,votos
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270


In [4]:
zomato.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [5]:
df = pd.DataFrame((zomato[['cozinha', 'taxa_votos', 'media_preco', 'alcance_preco', 'classifi_agregada', 'votos']]))
df.head(3)

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
0,"French, Japanese, Desserts",Excellent,1100,3,4.8,314
1,Japanese,Excellent,1200,3,4.5,591
2,"Seafood, Asian, Filipino, Indian",Very Good,4000,4,4.4,270


In [6]:
dfColunasParaDicionarizar = df[~df["cozinha"].str.contains(" ", na=False)]
print(dfColunasParaDicionarizar.head(3))

    cozinha taxa_votos  media_preco  alcance_preco  classifi_agregada  votos
1  Japanese  Excellent         1200              3                4.5    591
5   Chinese  Very Good         1000              3                4.4    336
9  Filipino  Excellent         1100              3                4.8    532


In [7]:
#dfColunasParaDicionarizar.isnull().sum()
dfColunasParaDicionarizar['cozinha'] = dfColunasParaDicionarizar['cozinha'].fillna("Brazilian")

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
dfColunasParaDicionarizar.isnull().sum()

cozinha              0
taxa_votos           0
media_preco          0
alcance_preco        0
classifi_agregada    0
votos                0
dtype: int64

In [15]:
dfColunasParaDicionarizar.fillna("Brazilian")

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
1,Japanese,Excellent,1200,3,4.5,591
5,Chinese,Very Good,1000,3,4.4,336
9,Filipino,Excellent,1100,3,4.8,532
13,Filipino,Very Good,1000,3,4.2,458
14,Korean,Very Good,700,3,4.3,223
17,Filipino,Excellent,1200,3,4.5,211
23,Cafe,Good,30,1,3.8,9
24,Bakery,Good,45,2,3.7,11
25,Brazilian,Good,60,3,3.8,11
26,Pizza,Average,50,2,3.2,11


In [9]:
taxa_votos = dfColunasParaDicionarizar['taxa_votos']
#print(dfColunasParaDicionarizar['cozinha'])

In [11]:
taxa_votos = taxa_votos.astype('str')
print(taxa_votos)

1       Excellent
5       Very Good
9       Excellent
13      Very Good
14      Very Good
17      Excellent
23           Good
24           Good
25           Good
26        Average
27        Average
28           Good
29           Good
31      Very Good
32        Average
35      Very Good
38        Average
39      Excellent
40        Average
46      Very Good
47      Excellent
51      Excellent
52      Very Good
53      Very Good
54      Excellent
56      Very Good
59      Excellent
64        Average
65      Very Good
69      Not rated
          ...    
9441    Very Good
9444    Excellent
9446         Good
9458    Excellent
9463    Very Good
9465    Excellent
9467    Very Good
9470    Very Good
9474    Very Good
9484    Excellent
9485    Very Good
9489    Very Good
9490    Very Good
9493    Very Good
9494    Very Good
9495    Very Good
9497      Average
9514    Excellent
9517    Very Good
9518    Excellent
9522    Excellent
9526    Very Good
9528    Very Good
9531         Good
9536    Ex

In [12]:
taxa_votos.isnull().sum()

0

In [1]:
ColunaTaxaVotos = dfColunasParaDicionarizar['taxa_votos']

NameError: name 'dfColunasParaDicionarizar' is not defined

In [29]:
ColunaTaxaVotos = ColunaTaxaVotos.astype('str')

In [31]:
X = taxa_votos
X = pd.concat([X, X], axis=1)
print(X.isnull().sum())

taxa_votos    0
taxa_votos    0
dtype: int64


In [34]:
print(X.shape)

(1679, 2)


In [32]:
labelencoder_dict = {}
onehotencoder_dict = {}

X_train = None
for i in range(0, X.shape[1]):
    label_encoder = preprocessing.LabelEncoder()
    labelencoder_dict[i] = label_encoder
    feature = label_encoder.fit_transform(X.iloc[:,i])
    #X_train.iloc[:,:]
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    onehotencoder_dict[i] = onehot_encoder
    if X_train is None:
          X_train = feature
    else:
          X_train = np.concatenate((X_train, feature), axis=1)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before

# Salvando dicionário

In [33]:
pickle.dump(labelencoder_dict, open('labelencoder_dict_para_cozinha.sav', 'wb'))

In [34]:
pickle.dump(onehotencoder_dict, open('onehotencoder_dict_para_cozinha.sav', 'wb'))

# Lendo arquivo de dicionário

In [38]:
labelencoder_dictArquivo = open('labelencoder_dict.sav', 'rb')

In [39]:
labelencoder_dictArquivo = open('onehotencoder_dict.sav', 'rb')

In [35]:
def getEncoded(test_data,labelencoder_dict,onehotencoder_dict):
    test_encoded_x = None
    for i in range(0,test_data.shape[1]):
        label_encoder =  labelencoder_dict[i]
        feature = label_encoder.transform(test_data.iloc[:,i])
        feature = feature.reshape(test_data.shape[0], 1)
        onehot_encoder = onehotencoder_dict[i]
        feature = onehot_encoder.transform(feature)
        if test_encoded_x is None:
              test_encoded_x = feature
        else:
              test_encoded_x = np.concatenate((test_encoded_x, feature), axis=1)
    return test_encoded_x

In [36]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
#X = df['cozinha']

In [38]:
dfProcessado = df[~df["cozinha"].str.contains(" ", na=False)]

In [41]:
dfColunasParaDicionarizar.head(2)

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
1,Japanese,Excellent,1200,3,4.5,591
5,Chinese,Very Good,1000,3,4.4,336


In [42]:
dfProcessado.head(2)

,cozinha,taxa_votos,media_preco,alcance_preco,classifi_agregada,votos
1,Japanese,Excellent,1200,3,4.5,591
5,Chinese,Very Good,1000,3,4.4,336


In [43]:
x_treinamentoParte1 = X_train
print(x_treinamentoParte1.shape)

(1679, 12)


In [44]:
treinamentoParte2 = pd.DataFrame((dfColunasParaDicionarizar[['media_preco','alcance_preco', 'classifi_agregada', 'votos']]))

In [45]:
treinamentoParte2.head(4)

,media_preco,alcance_preco,classifi_agregada,votos
1,1200,3,4.5,591
5,1000,3,4.4,336
9,1100,3,4.8,532
13,1000,3,4.2,458


In [46]:
x_treinamentoParte2 = np.array(treinamentoParte2)
print(x_treinamentoParte1.shape)
print(x_treinamentoParte2.shape)

(1679, 12)
(1679, 4)


In [47]:
X_TRAIN = np.concatenate((x_treinamentoParte1, x_treinamentoParte2), axis=1)

In [48]:
type(X_TRAIN)

numpy.ndarray

In [50]:
dfColunasParaDicionarizar.isnull().sum()

cozinha              0
taxa_votos           0
media_preco          0
alcance_preco        0
classifi_agregada    0
votos                0
dtype: int64

In [15]:
Y_TRAIN = np.array(dfColunasParaDicionarizar['cozinha'])

#pd.DataFrame((zomato[['cozinha', 'taxa_votos', 'media_preco', 'alcance_preco', 'classifi_agregada', 'votos']]))
#ly = pd.Dataframe((dfColunasParaDicionarizar['cozinha']))
# = dfColunasParaDicionarizar['cozinha']
s = pd.Series(dfColunasParaDicionarizar['cozinha'], dtype="category")
print(type(s))
print(s.head(4))

<class 'pandas.core.series.Series'>
1     Japanese
5      Chinese
9     Filipino
13    Filipino
Name: cozinha, dtype: category
Categories (72, object): [Afghani, African, American, Andhra, ..., Thai, Tibetan, Turkish, Vietnamese]


In [54]:
listacozinha = dfColunasParaDicionarizar["cozinha"].tolist() 
print(type(listacozinha))

<class 'list'>


In [55]:
cozinhasCSV = pd.DataFrame({'cozinhas': listacozinha})

In [58]:
print(type(cozinhasCSV))

<class 'pandas.core.frame.DataFrame'>


In [60]:
cozinhasCSV.to_csv('cozinhas.csv', sep=';', encoding='utf-8', index=False)

In [16]:
li = list(range(1, 72))
print(type(li))
y_train = s.cat.rename_categories(np.arange(72))

<class 'list'>


In [62]:
print(type(y_train))

<class 'pandas.core.series.Series'>


In [72]:
print(Y_TRAIN)

['Japanese' 'Chinese' 'Filipino' ... 'Cafe' 'Turkish' 'Cafe']


In [63]:
import xgboost as xgb
from sklearn import model_selection

In [64]:
xg_boost = xgb.XGBClassifier()

In [17]:
print(y_train)

1       40
5       22
9       27
13      27
14      45
17      27
23      19
24      10
25      15
26      59
27      40
28       4
29      40
31      39
32      59
35      37
38      62
39      37
40      39
46      46
47      18
51      15
52      15
53      39
54      40
56      15
59      15
64      35
65      15
69      15
        ..
9441    19
9444    50
9446    65
9458    66
9463     2
9465     1
9467    35
9470    51
9474    24
9484    28
9485    19
9489    51
9490    39
9493    51
9494    64
9495    50
9497    22
9514    62
9517    19
9518    43
9522    43
9526    64
9528    19
9531    19
9536    59
9537    25
9538    19
9543    19
9546    70
9550    19
Name: cozinha, Length: 1679, dtype: category
Categories (72, int64): [0, 1, 2, 3, ..., 68, 69, 70, 71]


In [115]:
y_train.to_csv('cozinhabinario.csv', index=False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [138]:
Y_TRAIN.shape
y_trainProFit = np.array(y_train)

media =y_train.fillna(1)
#df['Item_Weight'] = df['Item_Weight'].fillna((df['Item_Weight'].mean()))
#y_train.fillna(y_train.mean())

In [19]:
print(y_train.head(3))
print(type(y_train))

1    40
5    22
9    27
Name: cozinha, dtype: category
Categories (72, int64): [0, 1, 2, 3, ..., 68, 69, 70, 71]
<class 'pandas.core.series.Series'>


In [68]:
xg_boost.fit(X_TRAIN, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [70]:
print(xg_boost.score(X_TRAIN, y_train))

#scores = model_selection.cross_val_score(xg_boost, X_TRAIN, Y_TRAIN, cv=5, scoring='accuracy')
#print(scores)
#print("Kfold on XGBClassifier: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std()))

0.5402025014889815


In [133]:
# Realizando uma predição

In [72]:
escolha = ['Excellent', 1000, 4, 4, 600]

In [73]:
def escolhaParaVetorPredicao(escolhaLista):
    colunasCategoricasPredicao = [{'cozinha': escolhaLista[0]}, {'das': escolhaLista[1]}]
    colunasCategoricasPredicaoDic = pd.DataFrame(colunasCategoricasPredicao)
    return colunasCategoricasPredicaoDic

In [74]:
data = {'c1':[escolha[0]], 'c2':[escolha[0]]} 
  
# Create DataFrame 
datas = pd.DataFrame(data) 
print(datas)

          c1         c2
0  Excellent  Excellent


In [75]:
dfescolhaParaVetorPredicao = escolhaParaVetorPredicao(escolha)

In [76]:
print(dfescolhaParaVetorPredicao.shape)

(2, 2)


In [77]:
dicionarioCategoriasPredicaoParte1 = getEncoded(datas,labelencoder_dict,onehotencoder_dict)

In [78]:
dicionarioCategoriasPredicaoParte1.shape

(1, 12)

In [79]:
def concatenaParte1e2Dicionario(dicionarioCategoriasPredicaoParte1, escolha):
    PegandoParte2Escolha = np.asarray(escolha[1:])
    Parte2Escolha = PegandoParte2Escolha.reshape(1,4)
    escolha_para_predicao = np.concatenate((dicionarioCategoriasPredicaoParte1, Parte2Escolha), axis=1)
    
    return escolha_para_predicao

In [80]:
dfConcatenadoPraPredicao = concatenaParte1e2Dicionario(dicionarioCategoriasPredicaoParte1, escolha)

In [81]:
print(dfConcatenadoPraPredicao.shape)

(1, 16)


In [82]:
print("Média de preço para características buscadas: ", xg_boost.predict(dfConcatenadoPraPredicao))

Média de preço para características buscadas:  [39]


In [99]:
predicaovalor = xg_boost.predict(dfConcatenadoPraPredicao)

In [100]:
type(predicaovalor)

numpy.ndarray

In [104]:
dfColunasParaDicionarizar.isnull().sum()

cozinha              0
taxa_votos           0
media_preco          0
alcance_preco        0
classifi_agregada    0
votos                0
dtype: int64

In [108]:
print(dfColunasParaDicionarizar.loc[[5]])

   cozinha taxa_votos  media_preco  alcance_preco  classifi_agregada  votos
5  Chinese  Very Good         1000              3                4.4    336


In [117]:
print(dfColunasParaDicionarizar.loc[0])

KeyError: 0

In [110]:
def buscarestaurante(valorpredicao):
    for item in range(len(y_train)):
        if y_train.values[item] == [predicaovalor[0]]:
                return dfColunasParaDicionarizar.loc[[item]]

In [118]:
restaurante = buscarestaurante(predicaovalor[[0]])

In [113]:
print(restaurante)

     cozinha taxa_votos  media_preco  alcance_preco  classifi_agregada  votos
13  Filipino  Very Good         1000              3                4.2    458


In [114]:
print(y_train)

1       40
5       22
9       27
13      27
14      45
17      27
23      19
24      10
25      15
26      59
27      40
28       4
29      40
31      39
32      59
35      37
38      62
39      37
40      39
46      46
47      18
51      15
52      15
53      39
54      40
56      15
59      15
64      35
65      15
69      15
        ..
9441    19
9444    50
9446    65
9458    66
9463     2
9465     1
9467    35
9470    51
9474    24
9484    28
9485    19
9489    51
9490    39
9493    51
9494    64
9495    50
9497    22
9514    62
9517    19
9518    43
9522    43
9526    64
9528    19
9531    19
9536    59
9537    25
9538    19
9543    19
9546    70
9550    19
Name: cozinha, Length: 1679, dtype: category
Categories (72, int64): [0, 1, 2, 3, ..., 68, 69, 70, 71]


# Salvando o modelo do treinamento

In [158]:
pickle.dump(labelencoder_dict, open('labelencoder_dict.sav', 'wb'))

In [159]:
pickle.dump(onehotencoder_dict, open('onehotencoder_dict.sav', 'wb'))

In [71]:
pickle.dump(xg_boost, open('modelo_para_cozinha.pkl', 'wb'))

# Abrindo o modelo para classificação

In [164]:
labelencoder_dictArquivo = open('labelencoder_dict.sav', 'rb')

In [165]:
onehotencoder_dictArquivo = open('onehotencoder_dict.sav', 'rb')

In [166]:
labelencoder_dict_carregado = pickle.load(labelencoder_dictArquivo)

In [167]:
type(labelencoder_dict_carregado)

dict

In [154]:
arquivo = open('modeloTreinadoMoveMeTeste.pkl', 'rb')

In [155]:
modelo_carregado = pickle.load(arquivo)

In [156]:
numero = modelo_carregado.predict(dfConcatenadoPraPredicao)

# Realizando um predição com o modelo salvo

In [157]:
print("Média de preço predição carregada para características buscadas: ", modelo_carregado.predict(dfConcatenadoPraPredicao))

Média de preço predição carregada para características buscadas:  [170]
